<a href="https://colab.research.google.com/github/Ewallk/Fixacao/blob/main/SCRAPER_IMAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação das bibliotecas necessárias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime

### Bibliotecas para scraping

import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

/usr/local/lib/python3.8/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Definindo headers. Já disponível na própria documentação da biblioteca.

In [2]:
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", \
           "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", \
           "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

Fazendo o request do site.

In [3]:
r = requests.get('https://imapi.org/perfil/sao_luis-ma', headers=headers)

In [4]:
content = r.content

Fazendo o tratamento com o BeautifulSoup

In [5]:
soup = BeautifulSoup(content)

Vizualizando alguns dados da Cidade de São Luís-MA

In [6]:
labels = []
values = []
dims = []

In [7]:
for d in soup.findAll('article', attrs={'class':'dimensao-sticky'}):
  for d1 in d.findAll('div', attrs={'class': 'col-xs-3'}):
    label = d1.find('p', attrs={'class': 'label'})
    dado = d1.find('p', attrs={'class': 'dado'})
    print('Label: ', label.text)
    print('Dado: ', dado.text)
    print('----')

Label:  Região
Dado:  Nordeste
----
Label:  Porte da cidade
Dado:  Grande porte
----
Label:  Habitantes
Dado:  1.082.935
----
Label:  Nascimentos ao ano/ 1000 habitantes
Dado:  15.929
----


In [8]:
for d in soup.findAll('g'):
  if d.has_attr('data-nota'):
    print('Label: ', d['data-titulo'])
    print('Valor: ', d['data-nota'])

Label:  saúde
Valor:  40
Label:  nutrição
Valor:  42
Label:  cuidado responsivo
Valor:  52
Label:  aprendizagem inicial
Valor:  68
Label:  segurança e proteção
Valor:  25


In [9]:
for d in soup.findAll('section', attrs={'class':'row bottom-xs'}):
  for d1 in d.findAll('div', attrs={'class': 'col-xs-4 tl'}):
    dado = d1.find('p', attrs={'class': 'dado'})
    print('Dado: ', dado.text)
    label = d1.find('p', attrs={'class': 'label'})
    print('Label: ', label.text)

Dado:  2.951º lugar de 5.570
Label:  Ranking Brasil


Obtendo todos os indicadores contidos no site sobre a cidade e organizando-os em um Dataframe.

In [10]:
for s in soup.findAll('section', attrs={'class': 'card'}):
  for d in s.findAll('article', attrs={'class': 'indicador'}):
    dado = d.find('p', attrs={'class': 'dado'})
    label = d.find('p', attrs={'class': 'label'})
    
    print('Dado: ', dado.text)
    print('Label: ', label.text)
    
    municipio = d.find('div', attrs={'class': 'local ativo'})
    if municipio != None and municipio.has_attr('data-nota'):
      print('Valor: ', municipio['data-nota'])
      valor = municipio['data-nota']
    else:
      print('Valor: N/A')
      valor = 'N/A'
    print('----')
    dims.append(s['data-dimensao'])
    labels.append(label.text)
    values.append(valor)

Dado:  Início adequado do pré-natal
Label:  Percentual de gestantes que iniciaram o pré-natal com 12 semanas ou menos
Valor:  78,16
----
Dado:  Consultas de pré-natal
Label:  Percentual de gestantes com 6 ou mais consultas de pré‐natal 
Valor:  65,43
----
Dado:  Gravidez na adolescência
Label:  Percentual  de gravidez na adolescência
Valor:  16,46
----
Dado:  Cesáreas
Label:  Percentual de cesáreas
Valor:  54,10
----
Dado:  Prematuridade
Label:  Percentual de crianças nascidas com menos de 37 semanas completas de gestação
Valor:  13,10
----
Dado:  Baixo peso ao nascer
Label:  Percentual de nascidos vivos com menos de 2500g
Valor:  9,01
----
Dado:  Sífilis Congênita
Label:  Casos confirmados e notificados de sífilis congênita para cada 10.000 crianças  menores de 5 anos
Valor:  19,52
----
Dado:  Visitas domiciliares nos primeiros 10 dias de vida
Label:  Percentual de equipes da atenção básica que participaram do PMAQ 2º ciclo e fizeram visitas domiciliares nos primeiros 10 dias de vida 

In [11]:
res = {'dimensao': dims, 'label': labels, 'valor': values}

In [12]:
resultado_slz = pd.DataFrame().from_dict(res)
resultado_slz.head()

,dimensao,label,valor
0,saude,Percentual de gestantes que iniciaram o pré-na...,"78,16"
1,saude,Percentual de gestantes com 6 ou mais consulta...,"65,43"
2,saude,Percentual de gravidez na adolescência,"16,46"
3,saude,Percentual de cesáreas,"54,10"
4,saude,Percentual de crianças nascidas com menos de 3...,"13,10"


# Buscando dados de 10 dos municípios brasileiros

Utilizando o Selenium - Páginas Dinâmicas, devido ao uso de javascript no seletor de municípios do site.

In [13]:
#!pip install selenium
#!apt install chromium-chromedriver

### Código comentado, pois foi necessária a reinicialização do kernel para utilização correta da biblioteca recém instalada

In [14]:
from selenium import webdriver
from bs4 import BeautifulSoup

url = 'https://imapi.org/perfil/sao_luis-ma'

Configurações iniciais do Chrome para utilização do selenium.

In [15]:
#configurações presente na própria documentação da biblioteca

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [16]:
driver = webdriver.Chrome('chromedriver', options=chrome_options)

Rodando a URL

In [17]:
driver.get(url)

In [18]:
html = driver.page_source

Tratando os dados com o BeautifulSoup

In [19]:
soup2 = BeautifulSoup(html, 'html.parser')

In [20]:
municipios = [option.text for option in soup2.select('#municipios option')]

Adequando o código anteriormente criado, a todos os municipios.

In [21]:
### importando a biblioteca sleep para evitar a sobrecarga de solicitações no site

from time import sleep
from random import randint

In [22]:
labels = []
values = []
municipio = []
estado = []
dims = []

In [23]:
i = 1
for m in municipios[:10]:
  print(i, sep=' ', end=' ', flush=True)
  i = i + 1
  r = requests.get('https://www.imapi.org/perfil/'+m, headers=headers)
  content = r.content
  sleep(randint(1,2))
  soup_all = BeautifulSoup(content)
  for s in soup_all.findAll('section', attrs={'class': 'card'}):
    for d in s.findAll('article', attrs={'class': 'indicador'}):
      dado = d.find('p', attrs={'class': 'dado'})
      label = d.find('p', attrs={'class': 'label'})
            
      munic = d.find('div', attrs={'class': 'local ativo'})
      if munic != None and munic.has_attr('data-nota'):
        
        valor = munic['data-nota']
      else:
        valor = np.NaN
      municipio.append(m.split('-')[0])
      estado.append(m.split('-')[1])
      dims.append(s['data-dimensao'])
      labels.append(label.text)
      values.append(valor)

1 2 3 4 5 6 7 8 9 10 

In [27]:
res = {'estado':estado, 'municipio':municipio, 'dimensao':dims, 'label':labels, 'valor':values}

In [28]:
res_df = pd.DataFrame().from_dict(res)

In [30]:
res_df

,estado,municipio,dimensao,label,valor
0,go,abadia_de_goias,saude,Percentual de gestantes que iniciaram o pré-na...,"87,30"
1,go,abadia_de_goias,saude,Percentual de gestantes com 6 ou mais consulta...,"81,50"
2,go,abadia_de_goias,saude,Percentual de gravidez na adolescência,"21,50"
3,go,abadia_de_goias,saude,Percentual de cesáreas,"69,00"
4,go,abadia_de_goias,saude,Percentual de crianças nascidas com menos de 3...,"14,00"
...,...,...,...,...,...
305,sc,abdon_batista,seguranca,Casos notificados de violência para cada 1000...,"6,13"
306,sc,abdon_batista,seguranca,Casos notificados de violência para cada 1000...,"2,87"
307,sc,abdon_batista,seguranca,Percentual de famílias beneficiárias do Progra...,"54,32"
308,sc,abdon_batista,seguranca,"Estimativa da concentração diária de PM2,5 (ug...","4,84"
